In [1]:
import os
import requests
import time
import datetime
import pandas as pd

In [8]:
!

In [2]:
def fetch_aave_rates(rId, fromT, res):
    url = (f"https://aave-api-v2.aave.com/data/rates-history?"
           f"reserveId={rId}&from={fromT}&resolutionInHours={res}")
    try:
        response = requests.get(url)
        if response.status_code != 200:
            raise Exception(response.status_code);

        data = response.json()
        df = pd.DataFrame(data)
        df["date-time"] = df["x"].apply(lambda x: datetime.datetime(x["year"], x["month"] + 1, x["date"], x["hours"]))
        df.drop(columns=["x"], inplace=True)
        df = df[["date-time"] + [col for col in df.columns if col != "date-time"]]
        return df
        
    except Exception as e:
        raise Exception from e

In [3]:
def date_to_unixT(date_str):
    try:
        dt = datetime.datetime.strptime(date_str, "%d-%m-%Y")
        return int(time.mktime(dt.timetuple()))
    except Exception as e:
        print(e)

In [4]:
cryptos = [
    {
        "name": "Wrapped ETH", 
        "market": "Core Market", 
        "rId":"0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc20x2f39d218133AFaB8F2B819B1066c7E434Ad94E9e1"
    },
    {
        "name": "Wrapped BTC", 
        "market": "Core Market", 
        "rId":"0x2260fac5e5542a773aa44fbcfedf7c193bc2c5990x2f39d218133AFaB8F2B819B1066c7E434Ad94E9e1"
    },
    {
        "name": "Tether", 
        "market": "Core Market", 
        "rId":"0xdac17f958d2ee523a2206206994597c13d831ec70x2f39d218133AFaB8F2B819B1066c7E434Ad94E9e1"
    },
    {
        "name": "USD Coin", 
        "market": "Core Market", 
        "rId":"0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb480x2f39d218133AFaB8F2B819B1066c7E434Ad94E9e1"
    },
]

In [5]:
timeStamp = date_to_unixT("01-01-2000");
print(timeStamp)

946663200


In [8]:
for crypto in cryptos:
    file_name = f"{crypto['name']}_{crypto['market']}.csv"
    if os.path.exists(file_name):
        user_input = input(f"{file_name} already exists! Do you want to fetch again and replace the file? (y/n): ").strip().lower()
        
        if user_input == 'y':
            res = fetch_aave_rates(
                crypto["rId"],
                timeStamp,
                1,
            )
            res.to_csv(file_name, index=True)
            print(f"{file_name} has been replaced and saved.")
        else:
            print(f"Skipping {file_name}. No changes made.")
    else:
        res = fetch_aave_rates(
            crypto["rId"],
            timeStamp,
            1,
        )
        res.to_csv(file_name, index=True)
        print(f"{file_name} saved.")


Wrapped ETH_Core Market.csv already exists! Do you want to fetch again and replace the file? (y/n):  y


Wrapped ETH_Core Market.csv has been replaced and saved.
Wrapped BTC_Core Market.csv saved.
Tether_Core Market.csv saved.
USD Coin_Core Market.csv saved.
